In [ ]:
! pip install lakefs-client

  Using cached lakefs_client-1.44.0-py3-none-any.whl.metadata (41 kB)
Using cached lakefs_client-1.44.0-py3-none-any.whl (378 kB)

[notice] A new release of pip is available: 23.3.1 -> 25.1.1
[notice] To update, run: pip install --upgrade pip


In [8]:
import lakefs_client
from lakefs_client import Configuration
from lakefs_client.client import repositories_api, branches_api, objects_api
import pandas as pd
import numpy as np

In [ ]:
# Replace with your actual values
lakefs_endpoint = "http://localhost:8000"
access_key = "AKIA..."
secret_key = "abc123..."

configuration = Configuration()
configuration.username = access_key
configuration.password = secret_key
configuration.host = lakefs_endpoint

client = LakeFSClient(configuration)

In [9]:
def clean_data(raw_data):
    clean_data = raw_data.copy()
    # Remove not relevant columns
    clean_data = clean_data.dropna(subset=['region','age','weight','height','howlong','gender','eat', \
                               'train','background','experience','schedule','howlong', \
                               'deadlift','candj','snatch','backsq','experience',\
                               'background','schedule','howlong'])
    clean_data = clean_data.drop(columns=['affiliate','team','name','athlete_id','fran','helen','grace',\
                              'filthy50','fgonebad','run400','run5k','pullups','train'])

    # Remove Outliers

    clean_data = clean_data[clean_data['weight'] < 1500]
    clean_data = clean_data[clean_data['gender'] != '--']
    clean_data = clean_data[clean_data['age'] >= 18]
    clean_data = clean_data[(clean_data['height'] < 96) & (clean_data['height'] > 48)]

    clean_data = clean_data[(clean_data['deadlift'] > 0) & (clean_data['deadlift'] <= 1105)|((clean_data['gender'] == 'Female') \
                 & (clean_data['deadlift'] <= 636))]
    clean_data = clean_data[(clean_data['candj'] > 0) & (clean_data['candj'] <= 395)]
    clean_data = clean_data[(clean_data['snatch'] > 0) & (clean_data['snatch'] <= 496)]
    clean_data = clean_data[(clean_data['backsq'] > 0) & (clean_data['backsq'] <= 1069)]

    # Clean Survey Data

    decline_dict = {'Decline to answer|': np.nan}
    clean_data = clean_data.replace(decline_dict)
    clean_data = clean_data.dropna(subset=['background','experience','schedule','howlong','eat'])
    return clean_data

In [10]:
orig_data = pd.read_csv('athletes.csv')
orig_data.head(5)

,athlete_id,name,region,team,affiliate,gender,age,height,weight,fran,...,snatch,deadlift,backsq,pullups,eat,train,background,experience,schedule,howlong
0,2554.0,Pj Ablang,South West,Double Edge,Double Edge CrossFit,Male,24.0,70.0,166.0,NaN,...,NaN,400.0,305.0,NaN,NaN,I workout mostly at a CrossFit Affiliate|I hav...,I played youth or high school level sports|I r...,I began CrossFit with a coach (e.g. at an affi...,I do multiple workouts in a day 2x a week|,4+ years|
1,3517.0,Derek Abdella,NaN,NaN,NaN,Male,42.0,70.0,190.0,NaN,...,NaN,NaN,NaN,NaN,NaN,I have a coach who determines my programming|I...,I played youth or high school level sports|,I began CrossFit with a coach (e.g. at an affi...,I do multiple workouts in a day 2x a week|,4+ years|
2,4691.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,5164.0,Abo Brandon,Southern California,LAX CrossFit,LAX CrossFit,Male,40.0,67.0,NaN,211.0,...,200.0,375.0,325.0,25.0,I eat 1-3 full cheat meals per week|,I workout mostly at a CrossFit Affiliate|I hav...,I played youth or high school level sports|,I began CrossFit by trying it alone (without a...,I usually only do 1 workout a day|,4+ years|
4,5286.0,Bryce Abbey,NaN,NaN,NaN,Male,32.0,65.0,149.0,206.0,...,150.0,NaN,325.0,50.0,I eat quality foods but don't measure the amount|,I workout mostly at a CrossFit Affiliate|I inc...,I played college sports|,I began CrossFit by trying it alone (without a...,I usually only do 1 workout a day|I strictly s...,1-2 years|


In [14]:
print(orig_data.columns)
orig_data['deadlift']

Index(['athlete_id', 'name', 'region', 'team', 'affiliate', 'gender', 'age',
       'height', 'weight', 'fran', 'helen', 'grace', 'filthy50', 'fgonebad',
       'run400', 'run5k', 'candj', 'snatch', 'deadlift', 'backsq', 'pullups',
       'eat', 'train', 'background', 'experience', 'schedule', 'howlong'],
      dtype='object')


0         400.0
1           NaN
2           NaN
3         375.0
4           NaN
          ...  
423001      NaN
423002      NaN
423003      NaN
423004     80.0
423005      NaN
Name: deadlift, Length: 423006, dtype: float64